In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flower17/Participants_Data/sample_submission.csv
/kaggle/input/flower17/Participants_Data/Train.csv
/kaggle/input/flower17/Participants_Data/Test.csv


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.metrics import *
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import gc
from tqdm import *

In [2]:
train=pd.read_csv('/kaggle/input/flower17/Participants_Data/Train.csv')

In [3]:
test=pd.read_csv('/kaggle/input/flower17/Participants_Data/Test.csv')

In [5]:
ss=pd.read_csv('/kaggle/input/flower17/Participants_Data/sample_submission.csv')

In [6]:
df = pd.concat([train, test]).reset_index(drop = True)

In [7]:
df.nunique()

Area_Code        30900
Locality_Code       22
Region_Code       2185
Height             110
Diameter           151
Class                8
Species            263
dtype: int64

In [8]:
df['Region_Species'] = pd.factorize(df['Region_Code'].astype('str') + df['Species'].astype('str'))[0]
df['Region_Species'].nunique()

17667

In [9]:
df['height*diameter'] = df['Height'] * df['Diameter']
df['height/diameter'] = df['Height'] / df['Diameter']
df['log_diameter'] = np.log(df['Diameter'])

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
fts = ['Area_Code','Locality_Code','Region_Code','Height','Diameter', 'Species', 'Region_Species']

In [11]:
for f in fts:
  df[f + '_freq'] = df[f].map(df[f].value_counts(normalize = True))

In [12]:
cat_fts = ['Area_Code', 'Locality_Code', 'Region_Code', 'Species', 'Region_Species', 'Height', 'Diameter']
num_fts_2 = ['Height', 'Diameter','height*diameter', 'height/diameter', 'Locality_Code_freq', 'Species_freq']

for g in tqdm_notebook(cat_fts):

  num_fts = [c for c in num_fts_2 if c != g]
  grp = df.groupby(g)[num_fts].mean()
  grp.columns = [c + f'_grpd_by_{g}_mean' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].std()
  grp.columns = [c + f'_grpd_by_{g}_std' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].min()
  grp.columns = [c + f'_grpd_by_{g}_min' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

  grp = df.groupby(g)[num_fts].max()
  grp.columns = [c + f'_grpd_by_{g}_max' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [13]:
fts = ['Area_Code','Locality_Code','Region_Code','Height','Diameter', 'Species', 'Region_Species']
for g in tqdm_notebook(fts):
  rf = [c for c in fts if c != g]
  grp = df.groupby(g)[rf].nunique()
  grp.columns = [c + f'_grpd_by_{g}_nunique' for c in grp.columns]
  df = pd.merge(df, grp, on = g, how = 'left')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [14]:
train, test = df[:train.shape[0]].reset_index(drop = True), df[train.shape[0]:].reset_index(drop = True)

In [15]:
features = [c for c in train.columns if c not in ['Class', "Area_Code", "Region_Code", 'Region_Species']]

In [17]:
oofs = np.zeros((len(train), 8))
preds = np.zeros((len(test), 8))
N_SPLITS = 15
folds = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 77)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['Class'])):

  print(f'\n\n Fold {fold_} \\n\n')
  X_trn, y_trn = train[features].iloc[trn_idx], train["Class"][trn_idx]
  X_val, y_val = train[features].iloc[val_idx], train["Class"][val_idx]

  clf = LGBMClassifier(n_estimators = 10000, learning_rate = 0.05, colsample_bytree = 0.5, reg_alpha = 0.5, reg_lambda = 1, random_state = 2)
  #clf = CatBoostClassifier(iterations = 5000, learning_rate = 0.06, random_state=2, reg_lambda=0.5, eval_metric='MultiClass', task_type = 'GPU')

  clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], early_stopping_rounds = 150, verbose = 50)

  oofs[val_idx] = clf.predict_proba(X_val)
  preds += clf.predict_proba(test[features]) / N_SPLITS



 Fold 0 \n

Training until validation scores don't improve for 150 rounds
[50]	valid_0's multi_logloss: 0.886885
[100]	valid_0's multi_logloss: 0.793132
[150]	valid_0's multi_logloss: 0.768528
[200]	valid_0's multi_logloss: 0.758423
[250]	valid_0's multi_logloss: 0.753715
[300]	valid_0's multi_logloss: 0.75186
[350]	valid_0's multi_logloss: 0.750641
[400]	valid_0's multi_logloss: 0.753398
[450]	valid_0's multi_logloss: 0.75765
Early stopping, best iteration is:
[348]	valid_0's multi_logloss: 0.75064


 Fold 1 \n

Training until validation scores don't improve for 150 rounds
[50]	valid_0's multi_logloss: 0.860082
[100]	valid_0's multi_logloss: 0.773509
[150]	valid_0's multi_logloss: 0.750835
[200]	valid_0's multi_logloss: 0.742064
[250]	valid_0's multi_logloss: 0.741025
[300]	valid_0's multi_logloss: 0.742805
[350]	valid_0's multi_logloss: 0.744776
Early stopping, best iteration is:
[245]	valid_0's multi_logloss: 0.74067


 Fold 2 \n

Training until validation scores don't improve for

In [ ]:
oof_val_score = log_loss(train['Class'], oofs)
print(f'Final Log loss: {oof_val_score}')

In [ ]:
ss[ss.columns.tolist()] = preds